In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
import scipy.stats as st
import statsmodels.api as sm
from statsmodels.stats.proportion import proportions_ztest

# from textwrap import wrap

%matplotlib inline

# %run cleaning_join_all.ipynb
# df_join_clean = import_df_join_clean() # type: ignore

df_join_clean = pd.read_csv("../data/clean/df_join_clean.csv")

In [2]:
display(df_join_clean)

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,tenure,age_group,bal_category_3,bal_category_2,Variation
0,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:27:07,5,64,79,U,2,189023.86,1,4,New Client,Elderly,Medium,High,Test
1,9988021,580560515_7732621733,781255054_21935453173_531117,2,2017-04-17 15:26:51,5,64,79,U,2,189023.86,1,4,New Client,Elderly,Medium,High,Test
2,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:19:22,5,64,79,U,2,189023.86,1,4,New Client,Elderly,Medium,High,Test
3,9988021,580560515_7732621733,781255054_21935453173_531117,2,2017-04-17 15:19:13,5,64,79,U,2,189023.86,1,4,New Client,Elderly,Medium,High,Test
4,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:18:04,5,64,79,U,2,189023.86,1,4,New Client,Elderly,Medium,High,Test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319497,1574008,117364417_77840596075,528720790_71583064618_169151,0,2017-05-06 23:43:27,10,121,55,U,2,153238.83,3,6,Long Standing Client,Senior,Medium,High,Test
319498,2908510,814969699_90652851448,562606085_36368381773_92090,0,2017-05-10 22:57:17,21,252,34,M,3,141808.05,6,9,Long Standing Client,Adults,Medium,High,Control
319499,2908510,814969699_90652851448,562606085_36368381773_92090,2,2017-05-10 22:56:31,21,252,34,M,3,141808.05,6,9,Long Standing Client,Adults,Medium,High,Control
319500,2908510,814969699_90652851448,562606085_36368381773_92090,1,2017-05-10 22:56:23,21,252,34,M,3,141808.05,6,9,Long Standing Client,Adults,Medium,High,Control


In [3]:
# Sorts 
df_join_clean = df_join_clean.sort_values(by=["client_id", "visit_id", "date_time", "process_step" ]).reset_index(drop=True)
# df_join_clean

In [4]:
# Drops duplicates for step 0 and 4
df_join_clean[df_join_clean["process_step"] == 4].drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate confirms for same visit
df_join_clean[df_join_clean["process_step"] == 0].drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate starts for same visit

/var/folders/x5/xrbwdz1510vcx8w5dcdp2s7m0000gn/T/ipykernel_60683/3965299809.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_join_clean[df_join_clean["process_step"] == 4].drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate confirms for same visit
/var/folders/x5/xrbwdz1510vcx8w5dcdp2s7m0000gn/T/ipykernel_60683/3965299809.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_join_clean[df_join_clean["process_step"] == 0].drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate starts for same visit


## Considerations after combining the data:

In [5]:
# Evaluates funnel conversion

df_join_clean['steps_status'] = None
previous_visit_id = None
previous_value = None

for i in range(len(df_join_clean)):
    current_visit_id = df_join_clean.iloc[i]["visit_id"]
    current_value = df_join_clean.iloc[i]["process_step"]

    if current_value == 0:
        df_join_clean.iloc[i, df_join_clean.columns.get_loc('steps_status')] = 0
    else:
        if previous_value > current_value:
            df_join_clean.iloc[i, df_join_clean.columns.get_loc('steps_status')] = -1
        if previous_value == current_value:
            df_join_clean.iloc[i, df_join_clean.columns.get_loc('steps_status')] = 0
        if previous_value < current_value:
            df_join_clean.iloc[i, df_join_clean.columns.get_loc('steps_status')] = 1
        # if current_value == 4:
        #     df_join_clean.iloc[i, df_join_clean.columns.get_loc('steps_status')] = "confirm"
    
    previous_value = current_value

display(df_join_clean)

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,tenure,age_group,bal_category_3,bal_category_2,Variation,steps_status
0,555,402506806_56087378777,637149525_38041617439_716659,0,2017-04-15 12:57:56,3,46,30,U,2,25454.66,2,6,New Client,Young Adult,Low,Low,Test,0
1,555,402506806_56087378777,637149525_38041617439_716659,1,2017-04-15 12:58:03,3,46,30,U,2,25454.66,2,6,New Client,Young Adult,Low,Low,Test,1
2,555,402506806_56087378777,637149525_38041617439_716659,2,2017-04-15 12:58:35,3,46,30,U,2,25454.66,2,6,New Client,Young Adult,Low,Low,Test,1
3,555,402506806_56087378777,637149525_38041617439_716659,3,2017-04-15 13:00:14,3,46,30,U,2,25454.66,2,6,New Client,Young Adult,Low,Low,Test,1
4,555,402506806_56087378777,637149525_38041617439_716659,4,2017-04-15 13:00:34,3,46,30,U,2,25454.66,2,6,New Client,Young Adult,Low,Low,Test,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319497,9999729,834634258_21862004160,870243567_56915814033_814203,4,2017-05-08 16:09:40,10,124,31,F,3,107059.74,6,9,Long Standing Client,Adults,Medium,High,Test,1
319498,9999729,604429154_69247391147,99583652_41711450505_426179,0,2017-04-05 13:40:49,10,124,31,F,3,107059.74,6,9,Long Standing Client,Adults,Medium,High,Test,0
319499,9999729,604429154_69247391147,99583652_41711450505_426179,1,2017-04-05 13:41:04,10,124,31,F,3,107059.74,6,9,Long Standing Client,Adults,Medium,High,Test,1
319500,9999832,145538019_54444341400,472154369_16714624241_585315,0,2017-05-16 16:46:03,23,281,49,F,2,431887.61,1,4,Long Standing Client,Adults,Medium,High,Test,0


In [6]:
# Eveluates if steps are errors

df_join_clean['step_error'] = None
previous_visit_id = None
previous_value = None

for i in range(len(df_join_clean)):
    current_visit_id = df_join_clean.iloc[i]["visit_id"]
    current_value = df_join_clean.iloc[i]["process_step"]

    if current_value == 0:
        df_join_clean.iloc[i, df_join_clean.columns.get_loc('step_error')] = int(0)
    else:
        if previous_value > current_value:
            df_join_clean.iloc[i, df_join_clean.columns.get_loc('step_error')] = int(1)
        if previous_value == current_value:
            df_join_clean.iloc[i, df_join_clean.columns.get_loc('step_error')] = int(0)
        if previous_value < current_value:
            df_join_clean.iloc[i, df_join_clean.columns.get_loc('step_error')] = int(0)
        # if current_value == 4:
        #     df_join_clean.iloc[i, df_join_clean.columns.get_loc('step_error')] = "confirm"
    
    previous_value = current_value

display(df_join_clean)

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,tenure,age_group,bal_category_3,bal_category_2,Variation,steps_status,step_error
0,555,402506806_56087378777,637149525_38041617439_716659,0,2017-04-15 12:57:56,3,46,30,U,2,25454.66,2,6,New Client,Young Adult,Low,Low,Test,0,0
1,555,402506806_56087378777,637149525_38041617439_716659,1,2017-04-15 12:58:03,3,46,30,U,2,25454.66,2,6,New Client,Young Adult,Low,Low,Test,1,0
2,555,402506806_56087378777,637149525_38041617439_716659,2,2017-04-15 12:58:35,3,46,30,U,2,25454.66,2,6,New Client,Young Adult,Low,Low,Test,1,0
3,555,402506806_56087378777,637149525_38041617439_716659,3,2017-04-15 13:00:14,3,46,30,U,2,25454.66,2,6,New Client,Young Adult,Low,Low,Test,1,0
4,555,402506806_56087378777,637149525_38041617439_716659,4,2017-04-15 13:00:34,3,46,30,U,2,25454.66,2,6,New Client,Young Adult,Low,Low,Test,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319497,9999729,834634258_21862004160,870243567_56915814033_814203,4,2017-05-08 16:09:40,10,124,31,F,3,107059.74,6,9,Long Standing Client,Adults,Medium,High,Test,1,0
319498,9999729,604429154_69247391147,99583652_41711450505_426179,0,2017-04-05 13:40:49,10,124,31,F,3,107059.74,6,9,Long Standing Client,Adults,Medium,High,Test,0,0
319499,9999729,604429154_69247391147,99583652_41711450505_426179,1,2017-04-05 13:41:04,10,124,31,F,3,107059.74,6,9,Long Standing Client,Adults,Medium,High,Test,1,0
319500,9999832,145538019_54444341400,472154369_16714624241_585315,0,2017-05-16 16:46:03,23,281,49,F,2,431887.61,1,4,Long Standing Client,Adults,Medium,High,Test,0,0


In [7]:
# Cast floats as integers
df_join_clean = df_join_clean.astype({
    "steps_status": "int64", 
    "step_error": "int64"
    })

# df_join_clean

### Split the data in the control and test groups

In [8]:
# Split data into groups
df_test_group = df_join_clean[df_join_clean["Variation"] == "Test"]
df_control_group = df_join_clean[df_join_clean["Variation"] == "Control"]

# display(df_test_group)
# display(df_control_group)

### Sort the datasets by: client_id, visit_id (one customer can use the platform several times), process, and datetime


In [9]:
# # Required:
# # - change process_step to numeral (0 to 4)
# # - evaluate which floats to change to integers
# df_test_group = df_test_group.sort_values(by=["client_id", "visit_id", "process_step", "date_time"]).reset_index(drop=True)
# df_control_group = df_control_group.sort_values(by=["client_id", "visit_id", "process_step", "date_time"]).reset_index(drop=True)

In [10]:
# Check random client_ids test group

list_of_ids = df_test_group["client_id"].to_list()
df_test_group[df_test_group["client_id"] == random.choice(list_of_ids)][["client_id", "visit_id",  "date_time", "process_step", "steps_status", "step_error"]]

,client_id,visit_id,date_time,process_step,steps_status,step_error
189574,5953338,767152405_35364043434_986486,2017-03-15 11:17:13,0,0,0
189575,5953338,767152405_35364043434_986486,2017-03-15 11:17:16,1,1,0
189576,5953338,767152405_35364043434_986486,2017-03-15 11:23:01,2,1,0
189577,5953338,767152405_35364043434_986486,2017-03-15 11:23:13,0,0,0
189578,5953338,767152405_35364043434_986486,2017-03-15 11:27:35,0,0,0
189579,5953338,767152405_35364043434_986486,2017-03-15 11:27:37,1,1,0
189580,5953338,767152405_35364043434_986486,2017-03-15 11:27:52,2,1,0
189581,5953338,767152405_35364043434_986486,2017-03-15 11:29:44,3,1,0
189582,5953338,767152405_35364043434_986486,2017-03-15 11:30:14,4,1,0


In [11]:
df_control_group.dtypes

client_id             int64
visitor_id           object
visit_id             object
process_step          int64
date_time            object
clnt_tenure_yr        int64
clnt_tenure_mnth      int64
clnt_age              int64
gendr                object
num_accts             int64
bal                 float64
calls_6_mnth          int64
logons_6_mnth         int64
tenure               object
age_group            object
bal_category_3       object
bal_category_2       object
Variation            object
steps_status          int64
step_error            int64
dtype: object

In [21]:
# Completion Rate by Group

def get_completion_rate(df):
    num_visits = df["visit_id"].nunique()
    print("Number of Visits >> ", num_visits)

    num_confirm = df[df["process_step"] == 4]
    num_confirm.drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate confirms for same visit
    num_confirm = num_confirm["process_step"].count()
    print("Number of Confirms >> ", num_confirm)

    completion_rate = (num_confirm / num_visits).round(2)
    print("Completion Rate >> ", completion_rate)
    return completion_rate


In [13]:
print("Test Group")
get_completion_rate(df_test_group)
print("")

print("Control Group")
get_completion_rate(df_control_group)
print("")


Test Group
Number of Visits >>  36906
Number of Confirms >>  21583
Completion Rate >>  0.58

Control Group
Number of Visits >>  32020
Number of Confirms >>  15948
Completion Rate >>  0.5



/var/folders/x5/xrbwdz1510vcx8w5dcdp2s7m0000gn/T/ipykernel_60683/211576735.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_confirm.drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate confirms for same visit


In [14]:

print("Test Group bal > 1000000")
get_completion_rate(df_test_group[df_test_group["bal"] > 1000000])
print("")

print("Control Group bal > 1000000")
get_completion_rate(df_control_group[df_control_group["bal"] > 1000000])
print("")


Test Group bal > 1000000
Number of Visits >>  753
Number of Confirms >>  477
Completion Rate >>  0.63

Control Group bal > 1000000
Number of Visits >>  639
Number of Confirms >>  307
Completion Rate >>  0.48



/var/folders/x5/xrbwdz1510vcx8w5dcdp2s7m0000gn/T/ipykernel_60683/211576735.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_confirm.drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate confirms for same visit


In [15]:

print("Test Group bal > 1000000 and Male")
get_completion_rate(df_test_group[(df_test_group["bal"] > 1000000) & (df_test_group["gendr"] == "M")])
print("")

print("Control Group bal > 1000000 and Male")
get_completion_rate(df_control_group[(df_control_group["bal"] > 1000000) & (df_control_group["gendr"] == "M")])
print("")


Test Group bal > 1000000 and Male
Number of Visits >>  499
Number of Confirms >>  318
Completion Rate >>  0.64

Control Group bal > 1000000 and Male
Number of Visits >>  466
Number of Confirms >>  226
Completion Rate >>  0.48



/var/folders/x5/xrbwdz1510vcx8w5dcdp2s7m0000gn/T/ipykernel_60683/211576735.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_confirm.drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate confirms for same visit


In [16]:

print("Test Group bal > 1000000 and Adult Males")
get_completion_rate(df_test_group[(df_test_group["bal"] > 1000000) & (df_test_group["gendr"] == "M") & (df_test_group["age_group"] == "Adults") ])
print("")

print("Control Group bal > 1000000 and Adult Males")
get_completion_rate(df_control_group[(df_control_group["bal"] > 1000000) & (df_control_group["gendr"] == "M") & (df_control_group["age_group"] == "Adults") ])
print("")

Test Group bal > 1000000 and Adult Males
Number of Visits >>  64
Number of Confirms >>  43
Completion Rate >>  0.67

Control Group bal > 1000000 and Adult Males
Number of Visits >>  67
Number of Confirms >>  40
Completion Rate >>  0.6



/var/folders/x5/xrbwdz1510vcx8w5dcdp2s7m0000gn/T/ipykernel_60683/211576735.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_confirm.drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate confirms for same visit



- If a customer has several start times, only the last one must be considered.
- If a customer hits the confirmation step several times, consider the last one as the end of the process
- Keep in mind that this is a funnel process. Therefore, not all the customers who start the process will finish it and be able to proceed to the next one.
- There are lots of customers who participated in the experiment over a limited period of time. Some of them will be faster or more successful than others, but all of them count. Therefore, we're looking for "mean" times or rates!!!


## Hypothesis

In [22]:
#Null Hypothesis (H₀): There is no difference in error rates between Test and Control groups.
#Alternative Hypothesis (H₁): There is a significant difference in error rates.
# Given percentage error rates
error_rate_test = get_completion_rate(df_test_group)
error_rate_control = get_completion_rate(df_control_group)

# Sample sizes
N1 = df_test_group["visit_id"].nunique()   # Total samples in Test group
N2 = df_control_group["visit_id"].nunique()  # Total samples in Control group

N1, N2


Number of Visits >>  36906
Number of Confirms >>  21583
Completion Rate >>  0.58
Number of Visits >>  32020
Number of Confirms >>  15948
Completion Rate >>  0.5


/var/folders/x5/xrbwdz1510vcx8w5dcdp2s7m0000gn/T/ipykernel_60683/3797114380.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_confirm.drop_duplicates(subset=['visit_id', 'process_step'], keep="last", inplace=True) # Drop duplicate confirms for same visit


(36906, 32020)

In [24]:

# Convert percentages to counts
X1 = int(error_rate_test * N1)   # Errors in Test group
X2 = int(error_rate_control * N2) # Errors in Control group

# Perform proportion test
count = np.array([X1, X2])  # Number of errors
nobs = np.array([N1, N2])   # Total sample size

stat, p_value = proportions_ztest(count, nobs, alternative='two-sided')

print(f"Z-statistic: {stat}")
print(f"P-value: {p_value}")

# Decision Rule
alpha = 0.05  # Significance level
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference.")
else:
    print("Fail to reject the null hypothesis: No significant difference.")

Z-statistic: 21.02403633520168
P-value: 3.9534149195970854e-98
Reject the null hypothesis: There is a significant difference.




## Accordingly:

### Completion Rates
- Within the same group (control or test), are the completion rates (number of customers who reached the end of a step out of total who started) in each step the same? (... test)
- Between groups (control and test), are the completion rates (number of customers who reached the end of a step out of total who started) in each step the same? (... test)

### Error Rates
- Is the error rate (number of customers who had to go to a previous step out of total) in the test group smaller than the error rate in the control group (whatever error rate is) - 5%?
    - No, it's just 1%.

### Completion Times
- Within the same group (control or test), are the completion times of all the steps of the process the same? (... test)
- Between groups (control and test), are the completion times of each step the same? (...step)

### Segment Analysis for each rate analysis
- Do you find differences by age groups in control and test groups?

### Tableau
- Use Tableau to create graphs to summarize your findings.